<h1>함수적 자료구조</h1>
<br>
<li>함수적 = 순수함수 (부수효과 없는)</li>
<li>자료구조 = 자료를 효과적으로 관리할 수 있는 구조</li>
<br>
<p>함수적이기 때문에,</p>
<p>1. 데이터에 변화가 생길 때마다 새롭게 만들고</p>
<p>2. 한번 만들어진 데이터는 변하지않는다 (삭제 포함)</p>

In [14]:
val a=List(1,2,3,4)
val b=List(5,6,7,8)

val c=a++b

a: List[Int] = List(1, 2, 3, 4)
b: List[Int] = List(5, 6, 7, 8)
c: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8)

<p><b>Q) 그렇다면 불필요한 복사가 많이 일어나지않을까?</b></p>
<p><b>A) 복사는 일어나지 않는다. 구슬(데이터)이 변하지 않으므로(immutable), 구슬들의 순서만 바꿔서 엮어주면 된다</b></p>
<p> ( x - y - z ) a </p>
<p> ( x - z - a ) y </p>

<p> 구체적인 예제로 살펴보자<p>
<p> 다음은 Singly LinkedList 구조를 스칼라로 구현한 것이다. </p>
<p> 책에는 CList 가 아닌 List로 구현이 되어있는데, 본래 존재하는 List의 영향이 없음을 확실히 하기 위해 기존에 없는 CList로 작성하였다 </p>

<h1>0. 스칼라 문법체크 : 자료구조의 일반형 (자료생성자 + apply)</h1>
<br/>

In [10]:
sealed trait CList[+A]
 
case object Nil extends CList[Nothing]
 
case class Cons[A](head: A, tail: CList[A]) extends CList[A]
 
object CList {
 
  def apply[A](a: A*): CList[A] = {
    if (a.isEmpty) {
      Nil
    }else{
      Cons(a.head, apply(a.tail: _*))
    }
  }
}

defined trait CList
defined object Nil
defined class Cons
defined object CList

<p>우선 새로운 문법이 나왔으니 살펴보자</p>
<br/>
<li>1. sealed : 해당 파일 (.scala) 내에서 모든 구현이 선언되어 있어야 함을 뜻한다</li>
<li>2. [+A] : +는 A가 B의 하위타입일 때, CList[A]도 CList[B]의 하위타입이 되도록 만든다. 가변지정자라고 부른다(variance annotation)</li>
<li>3. apply : apply 메서드를 사용하면 CList(1,2,3,4) 등의 목록리터럴(list literal)방식으로 편하게 자료구조를 생성할 수 있게된다. </li>
<li>4. [a: A\*] : \*는 여러개의 인자(0개 이상)를 받을 수 있도록 만들어준다(variadic function). Seq의 syntatic sugar이다. a.head, a.tail 이 가능한 이유이다</li> 
<br/>


<p>case object Nil extends CList[Nothing]</p>
<p>case class Cons\[A\](head: A, tail: CList[A]) extends CList\[A\]</p>
<br/>
<p>을 <font color=red>자료 생성자(Data Constructor)</font> 라고 칭한다. 그냥 생성자와는 다르다. 구슬들을 실제로 엮는 역할을 맡고 있다</p>
<p>CList가 취할 수 있는 두 가지 형태를 표현한다</p>
<br/>
<li>1. Nil 은 비어있는 CList를</li>
<li>2. Cons는 내용물이 있는 CList를 표현한다</li>
<br/>
<p>이때, Cons의 tail은 Nil 일 수 있다</p>
<br/>
<p>Nil과 같은 형태가 가능한 이유는, 가변지정자(trait CList[+A]) 덕분에 CList[Nothing]이 CList[A]의 하위타입이 되었기 때문이다</p>
<p>참고로 Nothing은 모든 타입의 하위타입이다(Any는 모든 타입의 상위타입)</p>
<br/>
<p>이제 문법 체크가 끝났으니 다시 본론으로 돌아가 자료구조를 살펴보자</p>
<br/>

<h1>1. 재귀적 구슬(데이터) 엮기</h1>
<br/>

In [22]:
val list = CList(1,2,3,4,5)
println(list)

Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))


list: CList[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

<p>Cons 구조는 재귀적형태로 구성되어 있는데, </p>
<p>주목해야할 것은 </p>
<li>1. Cons의 구성이 원소(element) 한개와 : example) 1</li>
<li>2. 나머지 여러 데이터를 품고있는 Cons 로 이루어져 있다는 것이다 : example) Cons(2,Cons(3,Cons(hi,Cons(5,Nil))))</li>
<br/>
<p>이제 이러한 자료구조에서 구슬(자료)들을 다르게 엮는 여러 메서드들을 살펴보자</p>

In [35]:
def tail[A](l:CList[A]): CList[A] = l match {
    case Nil => throw new NoSuchElementException
    case Cons(x, xs) => xs
}
def drop[A](l:CList[A], n:Int): CList[A] = l match {
    case Nil => Nil
    case Cons(x, xs) => {
      if(n>1){
        drop(xs, n-1)
      }else{
        xs
      }
    }
}

def setHead[A](l:CList[A], n:A): CList[A] = l match {
    case Nil => CList(n)
    case Cons(x, xs) => Cons(n, xs)
}
def addHead[A](l:CList[A], n:A): CList[A] = l match {
    case Nil => CList(n)
    case Cons(x, xs) => Cons(n, Cons(x, xs))
}
def append[A](a1: CList[A], a2: CList[A]): CList[A] = a1 match {
    case Nil => a2
    case Cons(x, xs) => Cons(x, append(xs, a2))
}

defined function tail
defined function drop
defined function setHead
defined function addHead
defined function append

In [38]:
tail(list)
drop(list, 3)
drop(list, 6)
setHead(list, 9)
addHead(list, 9)
append(list, list)

res24_0: CList[Int] = Cons(2,Cons(3,Cons(4,Cons(5,Nil))))
res24_1: CList[Int] = Cons(4,Cons(5,Nil))
res24_2: CList[Int] = Nil
res24_3: CList[Int] = Cons(9,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))
res24_4: CList[Int] = Cons(9,Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil))))))
res24_5: CList[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil))))))))))

<p>한편, 자료구조의 각 요소들을 더한다거나, 곱하는 등의 조작도 당연히 가능하다</p>

In [43]:
def sum(ints: CList[Int]): Int = ints match {
    case Nil => 0
    case Cons(x, xs) => x + sum(xs)
}

def product(ds: CList[Int]): Int = ds match {
    case Nil => 1
    case Cons(x, xs) => x * product(xs)
}

defined function sum
defined function product

In [48]:
list
sum(list)
product(list)

res29_0: CList[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))
res29_1: Int = 15
res29_2: Int = 120

<h1>2. 고차 함수로의 일반화</h1>
<br/>
<p>그런데 sum과 product처럼 비슷하게 생긴 코드를 봤을 때, 일반화하고 싶은 마음이 들어야 프로그래머로써 옳은 자세다</p>
<br/>

In [51]:
def foldRight[A, B](l: CList[A], z: B)(f: (A, B) =>B): B = l match {
    case Nil => z
    case Cons(x, xs) => f(x, foldRight(xs, z)(f))
}

def sum2(l: CList[Int]) = foldRight(l, 0)((x,y)=> x + y)

def product2(l: CList[Int]) = foldRight(l, 1)(_ * _)

defined function foldRight
defined function sum2
defined function product2

<p>구현한 건 좋은데, 꼬리재귀가 아니니 stack safe 하지 않다. stack safe하게 다시 일반화해보자</p>

In [55]:
def foldLeft[A, B](l: CList[A], z: B)(f: (A, B) =>B): B = l match {
    case Nil => z
    case Cons(x, xs) => foldLeft(xs, f(x, z))(f)
}

def sum3(l: CList[Int]) = foldLeft(l, 0)((x,y)=> x + y)

def product3(l: CList[Int]) = foldLeft(l, 1)(_ * _)

defined function foldLeft
defined function sum3
defined function product3

In [56]:
sum2(list)
sum3(list)
product2(list)
product3(list)

res34_0: Int = 15
res34_1: Int = 15
res34_2: Int = 120
res34_3: Int = 120

<h1>3. 일반화의 단점 : 순회와 평가단축(5장을 기다리며...)</h1>
<br/>
<p><b>Q) 0을 곱하면 무조건 0이다. 일반화한 foldRight나 foldLeft로 만든 product2, product3는 중간에 0을 만났을 때 재귀를 멈추고 0을 돌려줄 수 있을까?</b></p>
<p><b>A) foldRight, Left는 무조건 전체 데이터를 순회하게되어있어서 평가단축이 불가능하다. 중간에 평가단축 로직을 넣자니 sum2, sum3가 망가진다</b></p>
<br/>
<h3> 그럼 어쩌나... 일반화를 포기해야하나?</h3>
<br/>

<h1>번외 : 커리함수와 타입추론</h1>
<br/>

In [59]:

 
  def findCharFirst(a: Array[Char], c: Char): Int = {
 
    def loop(n: Int): Int = {
      if (n >= a.length) {
        -1
      } else if (a(n) == c) {
        n
      } else {
        loop(n + 1)
      }
    }
    loop(0)
  }
 
  def findAnyFirst[T](a: Array[T], f: T => Boolean): Int = {
 
    def loop(n: Int): Int = {
      if (n >= a.length) {
        -1
      } else if (f(a(n))) {
        n
      } else {
        loop(n + 1)
      }
    }
    loop(0)
  }
 
  def findRealAnyFirst[T](a: Array[T])(f: T => Boolean): Int = {
 
    def loop(n: Int): Int = {
      if (n >= a.length) {
        -1
      } else if (f(a(n))) {
        n
      } else {
        loop(n + 1)
      }
    }
    loop(0)
  }

defined function findCharFirst
defined function findAnyFirst
defined function findRealAnyFirst

In [60]:
val targetChar = "Hello, world!".toCharArray
val targetInt = Array(1,2,3,4,5)
val targetFree = Array(1,2,3,"hi",5)
 
val res1 = findCharFirst(targetChar, 'r')
val res2 = findAnyFirst(targetInt, (x: Int) => x == 3)
val res3 = findRealAnyFirst(targetFree)(_ == "hi")
 
println(targetChar(res1).getClass+", "+targetChar(res1))
println(targetInt(res2).getClass+", "+targetInt(res2))
println(targetFree(res3).getClass+", "+targetFree(res3))

char, r
int, 3
class java.lang.String, hi


targetChar: Array[Char] = Array(
  'H',
  'e',
  'l',
  'l',
  'o',
  ',',
  ' ',
  'w',
  'o',
  'r',
  'l',
  'd',
  '!'
)
targetInt: Array[Int] = Array(1, 2, 3, 4, 5)
targetFree: Array[Any] = Array(1, 2, 3, hi, 5)
res1: Int = 9
res2: Int = 2
res3: Int = 3

<h3>1. findCharFirst 메서드에서는 고정된 type인 Char만을 다루지만</h3>
<br/>
<h3>2. findAnyFirst 메서드에서는 추상적 타입을 받아서 처리하고 있다</h3>
<br/>
<p>def findAnyFirst[T](a: Array[T], f: T => Boolean): Int</p>
<br/>
<p>parameter를 임의 type의 Array, 그리고 임의 type을 받는 함수로 지정해놓았음을 확인할 수 있다</p>
하지만 여전히 실사용시에는 type이 고정되어 있다 :  (x: Int) => x == 3</p>
<br/>
<h3>3. 반면 findRealAnyFirst 메서드에서는 진정한 type에 자유로운 모습을 보여주고 있다</h3>
<br/>
<p>def findRealAnyFirst[T](a: Array[T])(f: T => Boolean): Int</p>
<br/>
<p>이는 (a: Array[T])(f: T => Boolean)식으로 커링(currying) 되어있어 타입 추론을 할 수 있기 때문이다</p>
<br/>
<p>사실 findAnyFirst 메서드처럼 해도 타입을 추론할 수 있어야 맞는 거 같지만... </p>
<p>안타깝게도 안된다</p>
<br/>

